<a href="https://colab.research.google.com/github/jinsusong/NLP-BERT-Review/blob/main/BERT_Pretraining_%EB%B0%A9%EB%B2%95_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Training Instance로 Pretraining 시키는 과정

 Training Instance를 만드는 작업이 마무리되면 

In [ ]:
<입력 Sequence - 인코딩 됨>

 2      4    523  8 312   1   53 5234   4   323  123   3    21  33  22   21  123   3

<입력 Mask> (Padding을 마스킹하기 위해서)

1 ... 1 0 ... 0 (뒤에 0들은 padding 된 index들)


<마스킹 된 토큰의 라벨 - 인코딩 됨>

3444, 553


<마스킹 된 토큰의 위치>

1, 8


<Random Nex>

1 (True)


<Sequence ID> (해당 토큰이 Seq_A에 속하는지 Seq_B에 속하는지)

0 ... 0 1 ... 1
 

이제 Training Instance를 사용해서 BERT를 Pretraining 시킬 차례

### Embedding

BERT를 Pretraining 시키기 전에 Encoding 한 토큰을 Embedding 해야 함.

- Encoding을 통해 토큰을 구분되는 Index로 변화시킴

- 하지만 각 단어가 고유한 Index를 가지기 때문에 (One-hot Vector과 비슷하게), 단어 사이에 의미의 유사성이 없음.

- 단어를 Encoding만 하게 되면 모델로써는 두 단어가 어떤 관계에 있는지 알 수 없음.

- 이 문제를 해결하기 위해 나온 방법이 Embedding이다. 

- 이외에도 Embedding을 하는 이유는 굉장히 많다.



- BERT 는 세 종류의 Embedding을 거치는데 각각 아래와 같다. 

- 이해를 돕기 위해 Batch Size = 8 , Sequence Length = 128 인 상황을 가정

- Sequence Length :  [CLS] Seq_A [SEP] Seq_B [SEP]에 있는 토큰들의 개수

- hidden_layer_size = embedding size = 768 : 한 토큰을 Embedding 할 때 길이가 768인 벡터로 Embedding 하겠다는 뜻

Word Embedding  : Word Index Scalar > Embedding Vector

- torch.Size([8, 128]) > torch.Size([8, 128, 768])  / Layer Size : (32200, 768)

- 토큰의 Index (BERT의 경우 0 이상 32199 이하의 정수)가 길이가 768인 Vector로 바뀌는 Embedding 이다. 

- 따라서 스칼라가 벡터로 변환되었다. 물론 좀 더 자세히 보면, 스칼라가 One-hot Vecotr로 바뀌고, 그다음에 길이 768인 Vector로 전환되겠지만 말이다.

 

Position Embedding  : Position Index Scalar > Embedding Vector
- torch.Size([8, 128]) > torch.Size([8, 128, 768]) / Layer Size : (128, 768)

 

- BERT의 Input에는 해당 토큰의 위치가 어디인지에 대한 정보가 없다. 하지만 사람이 문장을 해석할 때 단어 위치는 굉장히 중요한 지표이므로, 이 정보를 추가해주어야 한다. 위 예시에서 Position Index Scalara 은 0~127 사이의 스칼라 값을 가질 것이다.

Token Type Embedding  : Token Type Scalar > Embedding Vector

- torch.Size([8, 128]) > torch.Size([8, 128, 768]) / Layer Size : (2, 768)

- 토큰 타입이랑 해당 토큰이 Seq_A 에 속하는지, Seq_B에 속하는지 에 대한 정보이다. A에 속하면 0, B에 속하면 1의 값을 가질 것이다. 따라서 이 또한 스칼라에서 벡터로의 Embedding이다. 

- 중요한 것은 세 Embedding 모두 Layer이라는 것이다. 

- 즉, Model이 학습할 때 그 대상이라는 것이다. PyTorch에서는 torch.nn.Embedding 모듈을 사용하며 아래 코드에서 확인할 수 있다. 

- torch.nn.Embedding의 모델 구조는 FFNN (Fully Connected)

In [ ]:
import torch.nn as nn


class BertEmbeddings(nn.Module):
    """Construct the embeddings from word, position and token_type embeddings.
    """
    
    def __init__(self, config):
        super(BertEmbeddings, self).__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size)  # (32200, 768)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)  # (128, 768)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)  # (2, 768)

        # self.LayerNorm is not snake-cased to stick with TensorFlow model variable name and be able to load
        # any TensorFlow checkpoint file
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)  # 0.1


    def forward(self, input_ids, token_type_ids):
        seq_length = input_ids.size(1)  # 128
        position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device) # torch.Size([128])
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids) # torch.Size([8, 128])

        words_embeddings = self.word_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)

        embeddings = words_embeddings + position_embeddings + token_type_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings  # (batchSize, sequenceLength, hidden_size)

In [ ]:
# 요약하면 
# Embedding Vector = Dropout(Normalize(Word Embedding + Position Embedding + Token Type Embedding))
 

### BERT Layer

 - BERT는 Transformer의 Encoder 를 직렬로 이은 구조를 갖는다.
 
 - BERT Block (Encoder Block), 혹은 BERT Layer 이라고 함. 
 
 - 한 Bert Layer은 세 가지 Layer로 구성되어 있다.
    -  BERT Attention
        1. Bert Self Attention (hidden_state, attention_mask)
        2. Bert Self Output (context, hidden_state)
    
    - BERT Intermediate

    - BERT Output

 

 BERT Attention

In [ ]:
# Bert Self Attention (hidden_state, attention_mask)

# 각각의 토큰에 대해서 Query, Key, Value를 만들고 Self Attention을 수행

# Multi Head Attentoin이기 때문에 768길이 Query, Key, Value를 12개의 64 길이로 나누는 것

 class BertSelfAttention(nn.Module):

    def __init__(self, config):
        super(BertSelfAttention, self).__init__()
        # hidden_size = 768, num_attention_heads = 12
        if config.hidden_size % config.num_attention_heads != 0:
            raise ValueError(
                "The hidden size (%d) is not a multiple of the number of attention "
                "heads (%d)" % (config.hidden_size, config.num_attention_heads))
        self.num_attention_heads = config.num_attention_heads  # 12
        self.attention_head_size = int(config.hidden_size / config.num_attention_heads)  # 64
        self.all_head_size = self.num_attention_heads * self.attention_head_size  # 768

        self.query = nn.Linear(config.hidden_size, self.all_head_size)  # (768, 768)
        self.key = nn.Linear(config.hidden_size, self.all_head_size)  # (768, 768)
        self.value = nn.Linear(config.hidden_size, self.all_head_size)  # (768, 768)

        self.dropout = nn.Dropout(config.attention_probs_dropout_prob)  # 0.1


    def transpose_for_scores(self, x):
        #  (batch_size, seq_length, num_attention_heads, attention_head_size)
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        # Multi-Head Self Attention
        # (batch_size, seq_length, all_head_size) -> (batch_size, seq_length, num_attention_heads, attention_head_size)
        x = torch.reshape(x, new_x_shape)
        
        return x.permute(0, 2, 1, 3)  # (batch_size, 12, 128, 64)


    def transpose_key_for_scores(self, x):
        new_x_shape = x.size()[:-1] + (self.num_attention_heads, self.attention_head_size)
        x = torch.reshape(x, new_x_shape)
        
        return x.permute(0, 2, 3, 1)   # (batch_size, 12, 64, 128)


    def forward(self, hidden_states, attention_mask):
        mixed_query_layer = self.query(hidden_states)  # (batchSize, 128, 768)
        mixed_key_layer = self.key(hidden_states)  # (batchSize, 128, 768)
        mixed_value_layer = self.value(hidden_states)  # (batchSize, 128, 768)

        query_layer = self.transpose_for_scores(mixed_query_layer)  # (batch_size, 12, 128, 64)
        key_layer = self.transpose_key_for_scores(mixed_key_layer)  # (batch_size, 12, 64, 128)
        value_layer = self.transpose_for_scores(mixed_value_layer)  # (batch_size, 12, 128, 64)

        # Take the dot product between "query" and "key" to get the raw attention scores.
        attention_scores = torch.matmul(query_layer, key_layer)  # (batch_size, 12, 128, 128)
        attention_scores = attention_scores / math.sqrt(self.attention_head_size)
        attention_scores = attention_scores + attention_mask 

        # Normalize the attention scores to probabilities.
        attention_probs = F.softmax(attention_scores, dim=-1)

        # This is actually dropping out entire tokens to attend to, which might
        # seem a bit unusual, but is taken from the original Transformer paper.
        attention_probs = self.dropout(attention_probs)

        context_layer = torch.matmul(attention_probs, value_layer)  # (batch_size, 12, 128, 64)
        context_layer = context_layer.permute(0, 2, 1, 3).contiguous()  # (batch_size, 128, 12, 64)
        new_context_layer_shape = context_layer.size()[:-2] + (self.all_head_size,)
        context_layer = torch.reshape(context_layer, new_context_layer_shape)  # (batch_size, 128, 768)
        
        return context_layer

In [ ]:
# Bert Self Output (context, hidden_state)

# Self Attention Layer을 거쳐 나온 Vecotr에 FFNN을 한 번 거치고, Skip Connection을 적용

class BertSelfOutput(nn.Module):

    def __init__(self, config):
        super(BertSelfOutput, self).__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)  # (768, 768)
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)


    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)  # (batch_size, 128, 768)
        hidden_states = self.dropout(hidden_states)  # (batch_size, 128, 768)
        hidden_states = self.LayerNorm(hidden_states + input_tensor) # Skip Connection
		
        return hidden_states

 

In [ ]:
# BERT Intermediate 

# Embedding Size 가 768이었던 Vecotr을 3072 (4배)로 늘려주는 FFNN을 거친다. Activation Function으로는 gelu를 사용

class BertIntermediate(nn.Module):

    def __init__(self, config):
        super(BertIntermediate, self).__init__()
        # (8, 128, 768) -> (8, 128, 3072)
        self.dense_act = LinearActivation(config.hidden_size, config.intermediate_size, act=config.hidden_act)

    def forward(self, hidden_states):
        hidden_states = self.dense_act(hidden_states)
        
        return hidden_states


In [ ]:
# BERT Output

# 이전에 3072로 늘어났던 Size를 다시 768로 바꾸고, Dropout & Normalize를 적용해준다. 여기서 나온 Output이 다음 Bert Layer의 Input으로 들어가게 된다. (다음 블록의 BERT Self Attention)

class BertOutput(nn.Module):

    def __init__(self, config):
        super(BertOutput, self).__init__()
        # (8, 128, 3072) -> (8, 128, 768)
        self.dense = nn.Linear(config.intermediate_size, config.hidden_size)
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, hidden_states, input_tensor):
        hidden_states = self.dense(hidden_states)
        hidden_states = self.dropout(hidden_states)
        hidden_states = self.LayerNorm(hidden_states + input_tensor)
        
        return hidden_states  # (8, 128, 768)


### BERT Pooler


- BERT 모델의 최종 Output으로 2가지를 내놓는다.

    1. tanh(마지막 hidden_state의 첫 토큰) -> -1~1 사이의 Scala 값 / Result : (8, 2)
        - 1번으로는 A와 B가 연속된 Sequence 인지 (Random Next) 판단하고,

    2. 마지막 hidden_state / Result : (8, 128, 32200)
        - 2번으로는 마스킹된 단어를 예측하게 된다.
        - 2번에서 길이 768 Vector을 32200 Vector로 바꾸게 되는데, 이때 layer으로는 Word Embedding Layer을 쓴다.

    
    

    

 



In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
from torch.nn import functional as F
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased',    return_dict = True)

text = "The capital of France, " + tokenizer.mask_token + ", contains the Eiffel Tower."
input = tokenizer.encode_plus(text, return_tensors = "pt")
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)
output = model(**input)
logits = output.logits
softmax = F.softmax(logits, dim = -1)
mask_word = softmax[0, mask_index, :]
top_10 = torch.topk(mask_word, 10, dim = 1)[1][0]
for token in top_10:
   word = tokenizer.decode([token])
   new_sentence = text.replace(tokenizer.mask_token, word)
   print(new_sentence)